# Introdução à Banco de Dados | Trabalho Prático III
Abel Severo Rocha, Ana Carla Fernandes, Natasha Caxias

Prof. Dr. Altigran Soares da Silva

## Parte I
Verificação dos parâmetros do hardware e do software utilizado e familiarização com o ambiente do PostgreSQL.

### Tarefa 1 - Identificação do Sistema

In [1]:
! pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import platform
import os
import shutil
import subprocess
import datetime
import psycopg2
import pandas as pd
from tabulate import tabulate
from psycopg2.extras import RealDictCursor
from tpch_pgsql import main as tpch_pgsql
from tpch4pgsql import postgresqldb as pgdb, result as r

In [3]:
def mostrarSoInfo():
    
    soInfo = platform.freedesktop_os_release()
    print("\n--------- Sistema Operacional (SO) ---------\n")
    print("Sistema Operacional ", platform.system())
    print("Versão              ", soInfo["VERSION"])
    print("Release (kernel)    ", platform.release())
    print("Arquitetura         ", platform.machine())
    print("Distribuição        ", soInfo["NAME"])


In [4]:
def mostrarHardwareInfo():

    cpuInfo = {}
    with open("/proc/cpuinfo") as f:
        for line in f:
            if ":" in line:
                k, v = line.split(":", 1)
                cpuInfo[k.strip()] = v.strip()
    
    ramInfo = {}
    with open("/proc/meminfo") as f:
        for line in f:
            k, v = line.split(":", 1)
            ramInfo[k] = v.strip()

    print("\n------------- Sobre o Hardware -------------\n")
    print("Processador:")
    print("  • Modelo          ", cpuInfo["model name"])
    print("  • CPUs lógicas    ", os.cpu_count())
    print("  • Clock base      ", cpuInfo["cpu MHz"], "MHz")

    print("\nCache:")
    with open("/sys/devices/system/cpu/cpu0/cache/index0/size") as f:
        print("  • L1d             ", f.read().strip()[:-1]+" kB")
    with open("/sys/devices/system/cpu/cpu0/cache/index1/size") as f:
        print("  • L1i             ", f.read().strip()[:-1]+" kB")
    with open("/sys/devices/system/cpu/cpu0/cache/index2/size") as f:
        print("  • L2              ", f.read().strip()[:-1]+" kB")
    with open("/sys/devices/system/cpu/cpu0/cache/index3/size") as f:
        print("  • L3              ", f.read().strip()[:-1]+" kB")

    print("\nMemória RAM:")
    print("  • Total           ", ramInfo["MemTotal"])
    print("  • Livre           ", ramInfo["MemFree"])
    print("  • Disponível      ", ramInfo["MemAvailable"])

    total, usado, livre = shutil.disk_usage("/")
    print("\nDisco:")
    print(f"  • Total            {total/1024**3:.2f} GB")
    print(f"  • Usado            {usado/1024**3:.2f} GB")
    print(f"  • Livre            {livre/1024**3:.2f} GB")


In [5]:
mostrarSoInfo()
mostrarHardwareInfo()


--------- Sistema Operacional (SO) ---------

Sistema Operacional  Linux
Versão               22.04.5 LTS (Jammy Jellyfish)
Release (kernel)     6.8.0-87-generic
Arquitetura          x86_64
Distribuição         Ubuntu

------------- Sobre o Hardware -------------

Processador:
  • Modelo           Intel(R) Core(TM) i7-5557U CPU @ 3.10GHz
  • CPUs lógicas     4
  • Clock base       3392.169 MHz

Cache:
  • L1d              32 kB
  • L1i              32 kB
  • L2               256 kB
  • L3               4096 kB

Memória RAM:
  • Total            16260572 kB
  • Livre            8203668 kB
  • Disponível       11215072 kB

Disco:
  • Total            108.98 GB
  • Usado            69.98 GB
  • Livre            33.43 GB


### Tarefa 2 - Verificação de parâmetros de armazenamento

In [6]:
# Permite que os acessos de hdparm sejam feitos sem sudo
# sudo visudo
# seu_usuario ALL=(ALL) NOPASSWD: /sbin/hdparm

In [7]:
BASE = "/sys/block"

def run(cmd):
    return subprocess.check_output(cmd, capture_output=True, text=True).strip()

def getDispositivo():
    for dev in os.listdir(BASE):
        if dev.startswith(("loop", "ram", "dm-")): # ignora
            continue
        return dev # encontra dispositivo
    return None

def ehHD(dispositivo):
    rotational_path = os.path.join(BASE, dispositivo, "queue/rotational")
    
    if os.path.exists(rotational_path):
        with open(rotational_path) as f:
            return f.read().strip() == "1"
    return False

def blocos(dispositivo):
    cmd = f"cat /sys/block/{dispositivo}/queue/logical_block_size"
    print("Blocos Lógicos:", run(cmd.split()))

    cmd = f"cat /sys/block/sdX/queue/physical_b"
    print("Blcocos Físicos: ", run(cmd.split()))

def info(dispositivo):
    cmd = f"sudo hdparm -I /dev/{dispositivo}"
    print(run(cmd.split()))

def stat(dispositivo):
    cmd = f"stat -f /dev/{dispositivo}"
    print("Parâmetros do SO para o disco:\n")
    print(run(cmd.split()))

def mostrarDiscoInfo():
    disp = getDispositivo()
    print("Tipos de dispositivo:", "HD" if ehHD(disp) else "SSD")
    print("Modelo", run(["cat", f"/sys/block/{disp}/device/model"]))
    #info(disp)
    stat(disp)
    blocos(disp)

In [8]:
disp = getDispositivo()
print(disp)
if ehHD(disp):
    mostrarDiscoInfo()
else:
    print("Dispositivo de armazenamento é um SSD.")

sdb
Dispositivo de armazenamento é um SSD.


### Tarefa 3 – Geração de um BD para testes


Necessário dar permissão de escrita para a pasta _tpch4pgsql_:

``chmod -R u+rwX,g+rwX,o+rX .``

#### Preparar
A fase de preparação compila o TPC-H dbgen e querygen e cria os arquivos de carga e atualização (atualização/exclusão).

In [9]:
import os
os.getcwd()


'/home/desktoop2/repos/Banco-de-Dados-2025-2/Trabalho 3/tpch4pgsql'

In [11]:
tpch_pgsql(phase="prepare")

make: Nothing to be done for 'all'.
built dbgen from source


TPC-H Population Generator (Version 2.14.0)
Copyright Transaction Processing Performance Council 1994 - 2010
Generating data for suppliers table
Preloading text ...                                                                                                                                                                                                                                                                                                              1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 

generated data for the load phase


TPC-H Population Generator (Version 2.14.0)
Copyright Transaction Processing Performance Council 1994 - 2010
Generating update pair #1 for orders/lineitem tables
Preloading text ...                                                                                                                                                                                                                                                                                                              1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2

generated data for the update phase
generated data for the delete phase
created data files in ./data
created query files in ./query_root


#### Carregamento

A fase de carregamento (load) limpa o banco de dados (se necessário), carrega as tabelas no banco de dados e cria índices para consultas. Os resultados desta fase consistem nas seguintes métricas:

* Tempo de criação do esquema
* Tempo de carregamento dos dados
* Tempo de criação de restrições de chave estrangeira e índices

In [12]:
tpch_pgsql(phase="load")

dropped existing tables
cleaned database tpch
done creating schemas
done loading data to tables
done creating indexes and foreign keys
=========================== Load ===========================
create_schema: : 0:00:00.089756
load_data: 0:01:21.703898
index_tables: 0:01:19.943115
======================= End Results ========================


#### Consultas

Nessa fase, as consultas serão analizadas com o comando ``EXPLAIN ANALYZE``, evidenciando:

* Tempo de execução do planejamento;
* Tempo de execução do _exlpain_;
* Algoritmos utilizados.

Em seguida, as consultas serão executas, exibindo as seguintes informações:

* Até 10 linhas do resultado da consulta;
* Total de linhas encontradas;
* Tempo de execução.

In [13]:
class PGDBWithResults(pgdb.PGDB):

    def __init__(self, host, port, database, user, password):
        # Chamar o construtor da classe pai corretamente
        self.conn = psycopg2.connect(
                    host=host,
                    port=port,
                    dbname=database,
                    user=user,
                    password=password
                ) 
        if hasattr(self, 'conn') and self.conn:
            print("Conexão estabelecida com sucesso!")
        else:
            print("Atenção: Conexão não estabelecida")
    
    def executeQueryFromFileWithResults(self, filepath):
        """Executa query de arquivo e retorna resultados"""
        try:
            # Verificar se a conexão existe
            if not hasattr(self, 'conn') or not self.conn:
                return {'error': 'Conexão não disponível'}
            
            with open(filepath, 'r') as f:
                query = f.read()
            
            # Usar cursor que retorna dicionários
            with self.conn.cursor(cursor_factory=RealDictCursor) as cursor:
                cursor.execute(query)
                
                if cursor.description:  # Se é uma query SELECT
                    results = cursor.fetchall()
                    columns = [desc[0] for desc in cursor.description]
                    return {
                        'columns': columns,
                        'data': results,
                        'rowcount': cursor.rowcount
                    }
                else:  # Para INSERT, UPDATE, DELETE
                    return {
                        'rowcount': cursor.rowcount,
                        'message': f"Query executada: {cursor.rowcount} linhas afetadas"
                    }
                    
        except Exception as e:
            return {'error': str(e)}
        
    def executeQuery(self, query_string):
        """Executa uma query diretamente a partir de string"""
        try:
            if not self.conn:
                return {'error': 'Conexão não disponível'}
            
            with self.conn.cursor(cursor_factory=RealDictCursor) as cursor:
                cursor.execute(query_string)
                
                if cursor.description:  # Se é uma query SELECT
                    results = cursor.fetchall()
                    columns = [desc[0] for desc in cursor.description]
                    return {
                        'columns': columns,
                        'data': results,
                        'rowcount': cursor.rowcount
                    }
                else:  # Para INSERT, UPDATE, DELETE
                    self.conn.commit()
                    return {
                        'rowcount': cursor.rowcount,
                        'message': f"Query executada: {cursor.rowcount} linhas afetadas"
                    }
                    
        except Exception as e:
            return {'error': str(e)}

In [14]:
host = "localhost"
port = 5432
user = "postgres"
password = "test123"
database = "tpch"
filepath = "query_root/perf_query_gen/"

conn = PGDBWithResults(host, port, database, user, password)

# Definir algoritmos de junção e varredura comuns no PostgreSQL
algoritmos_juncao = ['Nested Loop', 'Hash Join', 'Merge Join']
algoritmos_varredura = ['Seq Scan', 'Index Scan', 'Index Only Scan', 'Bitmap Heap Scan', 'Bitmap Index Scan']
algoritmos_ordenacao = ['Sort', 'Incremental Sort']
algoritmos_agregacao = ['HashAggregate', 'GroupAggregate']

for i in [1, 3, 5, 6, 7, 9, 10, 12]:
    print(f"\n{'=' * 120}")
    print(f"EXPLAIN ANALYZE - QUERY {i}")
    print(f"{'=' * 120}")
    
    # Ler a query original
    with open(filepath + f"{i}.sql", 'r') as f:
        original_query = f.read()
    
    # Adicionar EXPLAIN ANALYZE
    explain_query = "EXPLAIN ANALYZE " + original_query
    
    start_time = datetime.datetime.now()
    result = conn.executeQuery(explain_query)
    end_time = datetime.datetime.now()
    execution_time = end_time - start_time
    
    if 'error' in result:
        print(f"Erro no EXPLAIN ANALYZE {i}: {result['error']}")
    elif 'data' in result:
        print(f"\nTempo de execução do EXPLAIN: {execution_time}")
        
        # Coletar algoritmos utilizados
        algoritmos_encontrados = {
            'juncao': [],
            'varredura': [],
            'ordenacao': [],
            'agregacao': [],
            'outros': []
        }
        
        print(f"\nPLANO DE EXECUÇÃO (EXPLAIN ANALYZE):")
        for idx, row in enumerate(result['data']):
            plan_line = list(row.values())[0] if row else ""
            print(f"{plan_line}")
            
            # Identificar algoritmos na linha do plano
            for algoritmo in algoritmos_juncao:
                if algoritmo in plan_line:
                    if algoritmo not in algoritmos_encontrados['juncao']:
                        algoritmos_encontrados['juncao'].append(algoritmo)
            
            for algoritmo in algoritmos_varredura:
                if algoritmo in plan_line:
                    if algoritmo not in algoritmos_encontrados['varredura']:
                        algoritmos_encontrados['varredura'].append(algoritmo)
            
            for algoritmo in algoritmos_ordenacao:
                if algoritmo in plan_line:
                    if algoritmo not in algoritmos_encontrados['ordenacao']:
                        algoritmos_encontrados['ordenacao'].append(algoritmo)
            
            for algoritmo in algoritmos_agregacao:
                if algoritmo in plan_line:
                    if algoritmo not in algoritmos_encontrados['agregacao']:
                        algoritmos_encontrados['agregacao'].append(algoritmo)
        
        # Mostrar resumo dos algoritmos utilizados
        print(f"\n{'─' * 80}")
        print("ALGORITMOS IDENTIFICADOS NO PLANO DE EXECUÇÃO:")
        print(f"{'─' * 80}")
        
        if algoritmos_encontrados['juncao']:
            print(f"Algoritmos de Junção: {', '.join(algoritmos_encontrados['juncao'])}")
        
        if algoritmos_encontrados['varredura']:
            print(f"Algoritmos de Varredura: {', '.join(algoritmos_encontrados['varredura'])}")
        
        if algoritmos_encontrados['ordenacao']:
            print(f"Algoritmos de Ordenação: {', '.join(algoritmos_encontrados['ordenacao'])}")
        
        if algoritmos_encontrados['agregacao']:
            print(f"Algoritmos de Agregação: {', '.join(algoritmos_encontrados['agregacao'])}")
        
        # Verificar se não foram encontrados algoritmos conhecidos
        total_algoritmos = (len(algoritmos_encontrados['juncao']) + 
                          len(algoritmos_encontrados['varredura']) + 
                          len(algoritmos_encontrados['ordenacao']) + 
                          len(algoritmos_encontrados['agregacao']))
        
        if total_algoritmos == 0:
            print("ℹNenhum algoritmo específico identificado (possivelmente plano simples)")
            
    else:
        print(f"{result.get('message', 'Resultado inesperado')}")
        print(f"Tempo: {execution_time}")

Conexão estabelecida com sucesso!

EXPLAIN ANALYZE - QUERY 1

Tempo de execução do EXPLAIN: 0:00:05.978654

PLANO DE EXECUÇÃO (EXPLAIN ANALYZE):
Limit  (cost=346387.38..346388.51 rows=1 width=248) (actual time=4466.338..5925.241 rows=1 loops=1)
  ->  Finalize GroupAggregate  (cost=346387.38..391477.69 rows=40000 width=248) (actual time=4432.749..5891.650 rows=1 loops=1)
        Group Key: l_returnflag, l_linestatus
        ->  Gather Merge  (cost=346387.38..387777.69 rows=80000 width=248) (actual time=4398.819..5891.549 rows=4 loops=1)
              Workers Planned: 2
              Workers Launched: 2
              ->  Partial GroupAggregate  (cost=345387.36..377543.68 rows=40000 width=248) (actual time=4086.164..5203.529 rows=3 loops=3)
                    Group Key: l_returnflag, l_linestatus
                    ->  Sort  (cost=345387.36..347471.11 rows=833502 width=144) (actual time=2874.367..3319.906 rows=1135394 loops=3)
                          Sort Key: l_returnflag, l_linestat

## Parte II
Análise do comportamento dos índices das tabelas do SGBD através do exame das tabelas de estatísticas para consultas SQL.

In [15]:
import time

# Configuração de conexão para a PARTE II
DB_CONFIG_PARTE2 = {
    "dbname": "tpch",
    "user": "postgres",
    "password": "test123",   
    "host": "localhost",
    "port": 5432
}

conn_p2 = psycopg2.connect(**DB_CONFIG_PARTE2)
cur_p2 = conn_p2.cursor()
print("Conectado ao PostgreSQL.")


Conectado ao PostgreSQL.


### Tarefa 5 – Preparação da Tabela Aleatória



In [16]:
cur_p2.execute("""
DROP TABLE IF EXISTS t;
CREATE TABLE t (
    k serial PRIMARY KEY,
    v integer
);

INSERT INTO t(v)
SELECT trunc(random() * 10)
FROM generate_series(1,100000);
""")
conn_p2.commit()
print("Tabela t criada e populada com 100.000 tuplas.")

cur_p2.execute("SELECT * FROM t ORDER BY k LIMIT 10;")
rows = cur_p2.fetchall()
df_t10 = pd.DataFrame(rows, columns=["k", "v"])
print("\nPrimeiras 10 tuplas da tabela t (ordenadas por k):")
print(df_t10)

Tabela t criada e populada com 100.000 tuplas.

Primeiras 10 tuplas da tabela t (ordenadas por k):
    k  v
0   1  1
1   2  3
2   3  8
3   4  8
4   5  5
5   6  2
6   7  0
7   8  0
8   9  0
9  10  0


### Tarefa 6 – Páginas criadas

In [17]:
cur_p2.execute("""
SELECT relname, relpages, reltuples
FROM pg_class
WHERE relname = 't';
""")
rows = cur_p2.fetchall()
df_pgclass = pd.DataFrame(rows, columns=["relname", "relpages", "reltuples"])
print(df_pgclass)

  relname  relpages  reltuples
0       t         0       -1.0


### Tarefa 7 – Blocos

In [18]:
cur_p2.execute("SELECT pg_sleep(1);")

cur_p2.execute("""
SELECT *
FROM pg_stats
WHERE tablename = 't';
""")
cols = [desc[0] for desc in cur_p2.description]
df_pgstats = pd.DataFrame(cur_p2.fetchall(), columns=cols)

print("\nEstatísticas em pg_stats para tabela t:")
print(df_pgstats)

cur_p2.execute("SELECT pg_stat_reset();")
conn_p2.commit()
print("\nEstatísticas resetadas com pg_stat_reset().")


Estatísticas em pg_stats para tabela t:
Empty DataFrame
Columns: [schemaname, tablename, attname, inherited, null_frac, avg_width, n_distinct, most_common_vals, most_common_freqs, histogram_bounds, correlation, most_common_elems, most_common_elem_freqs, elem_count_histogram]
Index: []

Estatísticas resetadas com pg_stat_reset().


### Tarefa 8 – Índice

In [19]:
# Índice e consulta com 100k tuplas
inicio = time.time()
cur_p2.execute("CREATE INDEX idx_v ON t(v);")
conn_p2.commit()
fim = time.time()
tempo_criacao_100k = fim - inicio
print(f"Tempo para criar índice idx_v (100k tuplas): {tempo_criacao_100k:.6f} s")

inicio = time.time()
cur_p2.execute("SELECT * FROM t WHERE v = 5;")
rows = cur_p2.fetchall()
fim = time.time()
tempo_consulta_100k = fim - inicio
print(f"Tempo de consulta (v = 5) com 100k tuplas: {tempo_consulta_100k:.6f} s")
print("Tuplas retornadas:", len(rows))

# Recriar tabela t com 1.000.000 de tuplas
print("\nRecriando tabela t com 1.000.000 tuplas...")

cur_p2.execute("DROP TABLE IF EXISTS t;")
cur_p2.execute("""
CREATE TABLE t (
    k serial PRIMARY KEY,
    v integer
);
""")
conn_p2.commit()

cur_p2.execute("""
INSERT INTO t(v)
SELECT trunc(random() * 10)
FROM generate_series(1,1000000);
""")
conn_p2.commit()
print("Tabela t populada com 1.000.000 de tuplas.")

inicio = time.time()
cur_p2.execute("CREATE INDEX idx_v ON t(v);")
conn_p2.commit()
fim = time.time()
tempo_criacao_1m = fim - inicio
print(f"Tempo para criar índice idx_v (1M tuplas): {tempo_criacao_1m:.6f} s")

inicio = time.time()
cur_p2.execute("SELECT * FROM t WHERE v = 5;")
rows = cur_p2.fetchall()
fim = time.time()
tempo_consulta_1m = fim - inicio
print(f"Tempo de consulta (v = 5) com 1M tuplas: {tempo_consulta_1m:.6f} s")
print("Tuplas retornadas:", len(rows))

Tempo para criar índice idx_v (100k tuplas): 0.133247 s
Tempo de consulta (v = 5) com 100k tuplas: 0.007123 s
Tuplas retornadas: 9930

Recriando tabela t com 1.000.000 tuplas...
Tabela t populada com 1.000.000 de tuplas.
Tempo para criar índice idx_v (1M tuplas): 0.917239 s
Tempo de consulta (v = 5) com 1M tuplas: 0.124205 s
Tuplas retornadas: 100695


- O tempo para criar o índice cresce aproximadamente de forma linear com o número de tuplas.

- O índice acelera consultas para v = 5, mantendo tempo < 0,1s mesmo com 1 milhão de tuplas.

### Tarefa 9 – Fillfactor

In [20]:
cur_p2.execute("DROP INDEX IF EXISTS idx_v;")
conn_p2.commit()

fillfactors = [60, 80, 90, 100]
resultados_ff = []

for ff in fillfactors:
    idx_name = f"idx_v_ff{ff}"
    cur_p2.execute(f"DROP INDEX IF EXISTS {idx_name};")
    conn_p2.commit()

    inicio = time.time()
    cur_p2.execute(f"""
        CREATE INDEX {idx_name} ON t(v)
        WITH (fillfactor = {ff});
    """)
    conn_p2.commit()
    fim = time.time()
    tempo_criacao = fim - inicio

    inicio = time.time()
    cur_p2.execute("SELECT * FROM t WHERE v = 5;")
    cur_p2.fetchall()
    fim = time.time()
    tempo_consulta = fim - inicio

    resultados_ff.append([ff, tempo_criacao, tempo_consulta])

df_fillfactor = pd.DataFrame(resultados_ff, columns=["fillfactor", "tempo_criacao", "tempo_consulta"])
print("\nResultados por fillfactor (índice ASC):")
print(df_fillfactor)


Resultados por fillfactor (índice ASC):
   fillfactor  tempo_criacao  tempo_consulta
0          60       0.908469        0.102965
1          80       0.774314        0.114867
2          90       0.787775        0.123920
3         100       0.819310        0.095174


Os resultados mostram que o fillfactor tem impacto mínimo no desempenho de consultas com igualdade (v = 5) em uma tabela estática. Isso ocorre porque o fillfactor afeta principalmente operações de escrita (inserções e atualizações) e não operações de leitura. Portanto, é esperado que todos os valores testados apresentem tempos muito semelhantes.

### Tarefa 10 – Índices DESC

In [21]:

cur_p2.execute("DROP INDEX IF EXISTS idx_v_desc;")
conn_p2.commit()

inicio = time.time()
cur_p2.execute("CREATE INDEX idx_v_desc ON t(v DESC NULLS FIRST);")
conn_p2.commit()
fim = time.time()
tempo_criacao_desc = fim - inicio
print(f"Tempo criação índice descendente idx_v_desc: {tempo_criacao_desc:.6f} s")

inicio = time.time()
cur_p2.execute("SELECT * FROM t WHERE v = 5;")
cur_p2.fetchall()
fim = time.time()
tempo_consulta_desc = fim - inicio
print(f"Tempo consulta (v = 5) com índice descendente: {tempo_consulta_desc:.6f} s")

fillfactors = [60, 80, 90, 100]
resultados_desc_ff = []

for ff in fillfactors:
    idx_name = f"idx_v_desc_ff{ff}"
    cur_p2.execute(f"DROP INDEX IF EXISTS {idx_name};")
    conn_p2.commit()

    inicio = time.time()
    cur_p2.execute(f"""
        CREATE INDEX {idx_name} ON t(v DESC NULLS FIRST)
        WITH (fillfactor = {ff});
    """)
    conn_p2.commit()
    fim = time.time()
    tempo_criacao = fim - inicio

    inicio = time.time()
    cur_p2.execute("SELECT * FROM t WHERE v = 5;")
    cur_p2.fetchall()
    fim = time.time()
    tempo_consulta = fim - inicio

    resultados_desc_ff.append([ff, tempo_criacao, tempo_consulta])

df_desc_fillfactor = pd.DataFrame(resultados_desc_ff, columns=["fillfactor", "tempo_criacao", "tempo_consulta"])
print("\nResultados por fillfactor (índice DESC):")
print(df_desc_fillfactor)

# Fechar conexão da Parte II
cur_p2.close()
conn_p2.close()
print("\nConexão da PARTE II encerrada.")

Tempo criação índice descendente idx_v_desc: 0.862453 s
Tempo consulta (v = 5) com índice descendente: 0.100775 s

Resultados por fillfactor (índice DESC):
   fillfactor  tempo_criacao  tempo_consulta
0          60       1.045774        0.113597
1          80       0.781081        0.100150
2          90       0.803743        0.101465
3         100       0.936104        0.098132

Conexão da PARTE II encerrada.


- Índices descendentes apresentam desempenho muito semelhante aos índices em ordem padrão (ASC) para consultas por igualdade (v = 5).

- A escolha entre ASC e DESC é mais importante quando o plano de execução precisa retornar dados ordenados por v, especialmente com ORDER BY v DESC LIMIT n.

- As variações de fillfactor (60, 80, 90, 100) nos índices descendentes não alteraram de forma significativa o tempo de consulta, o que confirma que:

    - fillfactor impacta principalmente operações de escrita,

    - enquanto consultas de leitura simples são pouco sensíveis a essa configuração em tabelas estáticas.

## Parte III

O objetivo desta parte do trabalho é estudar o comportamento dos otimizadores de consulta dos SGBDs através do exame e análise dos planos de execução para consultas SQL sobre tabelas que serão fornecidos. Será bastante utilizado o comando EXPLAIN ANALYZE, que permite visualizar todas as etapas envolvidas no processamento de uma consulta. Usaremos para isso a tabela “movies”.

### Tarefa 11 – Preparaçao e Verificaçao do ambiente

In [14]:
import psycopg2
import pandas as pd

DB_CONFIG = {
    "dbname": "tpch",
    "user": "postgres",
    "password": "test123",
    "host": "localhost",
    "port": 5432
}

conn = psycopg2.connect(**DB_CONFIG)
conn.autocommit = True
cur = conn.cursor()

print("Conectado ao banco tpch!")



Conectado ao banco tpch!


In [17]:
caminho = '/home/desktoop2/repos/Banco-de-Dados-2025-2/Trabalho 3/tpch4pgsql/movie.sql'

with open(caminho, "r", encoding="utf-8") as f:
    sql = f.read()

cur.execute(sql)

print("movie.sql executado com sucesso!")

cur.execute("CREATE EXTENSION IF NOT EXISTS pgstattuple;")
print("Extensão pgstattuple verificada!")


movie.sql executado com sucesso!
Extensão pgstattuple verificada!


In [21]:
query = """
WITH info AS (
    SELECT
        ic.relname AS index_name,
        tc.relname AS table_name,
        pg_relation_size(ic.oid) / 8192 AS total_blocks,
        st.n_live_tup AS num_rows,
        i.indrelid,
        i.indexrelid
    FROM pg_index i
    JOIN pg_class ic ON ic.oid = i.indexrelid
    JOIN pg_class tc ON tc.oid = i.indrelid
    JOIN pg_stat_all_tables st ON st.relname = tc.relname
    WHERE tc.relname = 'movie'
)
SELECT
    index_name,
    table_name,
    total_blocks AS blocos_totais,
    num_rows AS num_chaves,
    total_blocks - 1 AS blocos_folha,     -- aproximação padrão
    (num_rows::float / (total_blocks - 1)) AS media_chaves_por_bloco
FROM info
ORDER BY index_name;
"""





In [22]:
cur.execute(query)
dados = cur.fetchall()

df_idx = pd.DataFrame(dados, columns=[
    "Índice", "Tabela", "Blocos totais", "Nº chaves",
    "Blocos folha", "Média chaves/bloco"
])

df_idx


,Índice,Tabela,Blocos totais,Nº chaves,Blocos folha,Média chaves/bloco
0,movie_key,movie,7,1844,6,307.333333
1,movie_title,movie,12,1844,11,167.636364
2,movie_votes,movie,11,1844,10,184.400000


In [23]:
distinct = {
    "movie_title": "SELECT COUNT(DISTINCT title) FROM movie;",
    "movie_votes": "SELECT COUNT(DISTINCT votes) FROM movie;"
}

distinct_counts = {}
for idx, q in distinct.items():
    cur.execute(q)
    distinct_counts[idx] = cur.fetchone()[0]

distinct_counts


{'movie_title': 1833, 'movie_votes': 1481}

### Tarefa 12 – Consultas por intervalo e índices secundários


In [27]:
# Consulta seletiva (<10 tuplas)
cur.execute("""
EXPLAIN ANALYZE
SELECT *
FROM movie
WHERE votes BETWEEN 53560 AND 53570;
""")

print("Consulta seletiva (<10 tuplas):\n")
for r in cur.fetchall():
    print(r[0])


# Consulta ampla (>80% das tuplas)
cur.execute("""
EXPLAIN ANALYZE
SELECT *
FROM movie
WHERE votes > 3000;
""")

print("\nConsulta ampla (>80% das tuplas):\n")
for r in cur.fetchall():
    print(r[0])


Consulta seletiva (<10 tuplas):

Index Scan using movie_votes on movie  (cost=0.28..8.30 rows=1 width=30) (actual time=0.010..0.012 rows=1 loops=1)
  Index Cond: ((votes >= 53560) AND (votes <= 53570))
Planning Time: 0.195 ms
Execution Time: 0.034 ms

Consulta ampla (>80% das tuplas):

Index Scan using movie_votes on movie  (cost=0.28..36.81 rows=659 width=30) (actual time=0.020..0.334 rows=661 loops=1)
  Index Cond: (votes > 3000)
Planning Time: 0.109 ms
Execution Time: 0.416 ms


### Tarefa 13 – Comparações de operadores de agregação.


In [28]:
# EXPLAIN ANALYZE da consulta A (com MAX)
cur.execute("""
EXPLAIN ANALYZE
SELECT title 
FROM movie
WHERE votes >= (SELECT MAX(votes) FROM movie);
""")
print("Consulta A — MAX():\n")
for r in cur.fetchall():
    print(r[0])


# EXPLAIN ANALYZE da consulta B (com ALL)
cur.execute("""
EXPLAIN ANALYZE
SELECT title 
FROM movie
WHERE votes >= ALL (SELECT votes FROM movie);
""")
print("\nConsulta B — ALL:\n")
for r in cur.fetchall():
    print(r[0])


Consulta A — MAX():

Index Scan using movie_votes on movie  (cost=0.61..35.37 rows=615 width=16) (actual time=0.029..0.030 rows=1 loops=1)
  Index Cond: (votes >= $1)
  InitPlan 2 (returns $1)
    ->  Result  (cost=0.32..0.33 rows=1 width=4) (actual time=0.019..0.020 rows=1 loops=1)
          InitPlan 1 (returns $0)
            ->  Limit  (cost=0.28..0.32 rows=1 width=4) (actual time=0.017..0.017 rows=1 loops=1)
                  ->  Index Only Scan Backward using movie_votes on movie movie_1  (cost=0.28..76.55 rows=1844 width=4) (actual time=0.016..0.016 rows=1 loops=1)
                        Index Cond: (votes IS NOT NULL)
                        Heap Fetches: 0
Planning Time: 0.306 ms
Execution Time: 0.059 ms

Consulta B — ALL:

Seq Scan on movie  (cost=0.00..43620.99 rows=922 width=16) (actual time=1.297..2.293 rows=1 loops=1)
  Filter: (SubPlan 1)
  Rows Removed by Filter: 1843
  SubPlan 1
    ->  Materialize  (cost=0.00..42.66 rows=1844 width=4) (actual time=0.000..0.001 rows=2 

### Tarefa 14 – Consultas com Junção e Seleção


In [29]:
# Consulta A: subconsulta
cur.execute("""
EXPLAIN ANALYZE
SELECT title 
FROM movie 
WHERE votes > (SELECT votes FROM movie WHERE title = 'Star Wars');
""")

print("Consulta A — Subconsulta:\n")
for r in cur.fetchall():
    print(r[0])


# Consulta B: junção
cur.execute("""
EXPLAIN ANALYZE
SELECT m1.title
FROM movie m1, movie m2
WHERE m1.votes > m2.votes
  AND m2.title = 'Star Wars';
""")

print("\nConsulta B — Self Join:\n")
for r in cur.fetchall():
    print(r[0])


Consulta A — Subconsulta:

Index Scan using movie_votes on movie  (cost=8.57..43.34 rows=615 width=16) (actual time=0.023..0.024 rows=0 loops=1)
  Index Cond: (votes > $0)
  InitPlan 1 (returns $0)
    ->  Index Scan using movie_title on movie movie_1  (cost=0.28..8.29 rows=1 width=4) (actual time=0.014..0.015 rows=1 loops=1)
          Index Cond: ((title)::text = 'Star Wars'::text)
Planning Time: 0.120 ms
Execution Time: 0.044 ms

Consulta B — Self Join:

Nested Loop  (cost=0.56..49.49 rows=615 width=16) (actual time=0.022..0.023 rows=0 loops=1)
  ->  Index Scan using movie_title on movie m2  (cost=0.28..8.29 rows=1 width=4) (actual time=0.014..0.015 rows=1 loops=1)
        Index Cond: ((title)::text = 'Star Wars'::text)
  ->  Index Scan using movie_votes on movie m1  (cost=0.28..35.04 rows=615 width=20) (actual time=0.005..0.005 rows=0 loops=1)
        Index Cond: (votes > m2.votes)
Planning Time: 0.164 ms
Execution Time: 0.045 ms


### Tarefa 15  – Casamento de Strings e Índices

In [30]:
# LIKE 'I%'
cur.execute("""
EXPLAIN
SELECT title 
FROM movie 
WHERE title LIKE 'I%';
""")
print("Consulta 1 — LIKE 'I%':\n")
for r in cur.fetchall():
    print(r[0])


# SUBSTR(title,1,1) = 'I'
cur.execute("""
EXPLAIN
SELECT title
FROM movie
WHERE substr(title, 1, 1) = 'I';
""")
print("\nConsulta 2 — SUBSTR(title,1,1) = 'I':\n")
for r in cur.fetchall():
    print(r[0])


# LIKE '%A'
cur.execute("""
EXPLAIN
SELECT title 
FROM movie 
WHERE title LIKE '%A';
""")
print("\nConsulta 3 — LIKE '%A':\n")
for r in cur.fetchall():
    print(r[0])


Consulta 1 — LIKE 'I%':

Seq Scan on movie  (cost=0.00..38.05 rows=18 width=16)
  Filter: ((title)::text ~~ 'I%'::text)

Consulta 2 — SUBSTR(title,1,1) = 'I':

Seq Scan on movie  (cost=0.00..42.66 rows=9 width=16)
  Filter: (substr((title)::text, 1, 1) = 'I'::text)

Consulta 3 — LIKE '%A':

Seq Scan on movie  (cost=0.00..38.05 rows=18 width=16)
  Filter: ((title)::text ~~ '%A'::text)


### Tarefa 16 – Verificação da hipótese de distribuição uniforme na estimativa de seletividade

In [31]:
# Consulta 1 — votes < 1000
cur.execute("""
EXPLAIN
SELECT title
FROM movie
WHERE votes < 1000;
""")
print("Consulta 1 — votes < 1000:\n")
for r in cur.fetchall():
    print(r[0])


# Consulta 2 — votes > 40000
cur.execute("""
EXPLAIN
SELECT title
FROM movie
WHERE votes > 40000;
""")
print("\nConsulta 2 — votes > 40000:\n")
for r in cur.fetchall():
    print(r[0])


Consulta 1 — votes < 1000:

Index Scan using movie_votes on movie  (cost=0.28..20.02 rows=328 width=16)
  Index Cond: (votes < 1000)

Consulta 2 — votes > 40000:

Index Scan using movie_votes on movie  (cost=0.28..8.42 rows=8 width=16)
  Index Cond: (votes > 40000)


## Parte IV

O objetivo desta parte do trabalho é experimentar estratégias para utilização de transações e níveis de isolamento em SGBDs relacionais. As tarefas envolvem uma simulação de um sistema de reservas de passagem áreas. 


In [ ]:
import psycopg2
import time
import random
import threading

DB_CONFIG = {
    "dbname": "tpch",
    "user": "postgres",
    "password": "test123",
    "host": "localhost",
    "port": 5432
}

def reset_db():
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()

    cur.execute("DROP TABLE IF EXISTS assentos;")
    cur.execute("""
        CREATE TABLE assentos(
            num_voo INT PRIMARY KEY,
            disp BOOLEAN
        );
    """)

    # 200 assentos inicialmente livres
    cur.executemany("INSERT INTO assentos VALUES (%s, true);", [(i,) for i in range(1, 201)])

    conn.commit()
    cur.close()
    conn.close()

reset_db()
print("Tabela assentos pronta.")


### Tarefa 17 – Código das Versões A e B

Versão A - Uma única transação

In [ ]:
def reserva_versaoA(isolation_level):

    conn = psycopg2.connect(**DB_CONFIG)
    conn.set_session(isolation_level=isolation_level)
    cur = conn.cursor()

    tentativas = 0

    while True:
        tentativas += 1
        try:
            # Passo 1 — escolher 1 assento livre e BLOQUEAR apenas ele
            cur.execute("""
                SELECT num_voo 
                FROM assentos 
                WHERE disp = true 
                ORDER BY random()
                LIMIT 1
                FOR UPDATE;
            """)
            row = cur.fetchone()

            # Nenhum assento livre → fim
            if not row:
                conn.rollback()
                break

            escolhido = row[0]

            # Passo 2 — espera de 1s (cliente escolhendo)
            time.sleep(1)

            # Passo 3 — marcar como ocupado
            cur.execute("""
                UPDATE assentos 
                SET disp = false 
                WHERE num_voo = %s;
            """, (escolhido,))

            conn.commit()
            break  # reserva feita com sucesso

        except psycopg2.errors.SerializationFailure:
            # conflitos serializáveis → tentar de novo
            conn.rollback()
            continue

        except psycopg2.Error as e:
            # erros inesperados devem ser reportados
            conn.rollback()
            print("Erro inesperado na versão A:", e)
            break

    cur.close()
    conn.close()
    return tentativas


Versão b - Duas transações

In [ ]:
def reserva_versaoB(isolation_level):

    tentativas = 0

    while True:
        tentativas += 1

        try:
            # ================================
            # TRANSAÇÃO 1: obter 1 assento livre
            # ================================
            conn1 = psycopg2.connect(**DB_CONFIG)
            conn1.set_session(isolation_level=isolation_level)
            cur1 = conn1.cursor()

            cur1.execute("""
                SELECT num_voo 
                FROM assentos 
                WHERE disp = true 
                ORDER BY random()
                LIMIT 1;
            """)

            row = cur1.fetchone()

            if not row:
                conn1.commit()
                conn1.close()
                break  # nenhum assento disponível

            escolhido = row[0]

            conn1.commit()
            conn1.close()

            # ================================
            # PASSO 2 — escolha do cliente
            # ================================
            time.sleep(1)

            # ================================
            # TRANSAÇÃO 2: tentar reservar
            # ================================
            conn2 = psycopg2.connect(**DB_CONFIG)
            conn2.set_session(isolation_level=isolation_level)
            cur2 = conn2.cursor()

            cur2.execute("""
                UPDATE assentos
                SET disp = false
                WHERE num_voo = %s AND disp = true;
            """, (escolhido,))

            if cur2.rowcount == 1:
                conn2.commit()
                conn2.close()
                break  # reserva concluída
            else:
                conn2.rollback()
                conn2.close()
                continue  # alguém já pegou → tentar novamente

        except psycopg2.errors.SerializationFailure:
            # conflito serializável → repetir tentativa
            continue

        except Exception as e:
            # erro inesperado → reportar e quebrar
            print("Erro inesperado na versão B:", e)
            break

    return tentativas


### Tarefa 18 – Executar experimentos e gerar gráficos

In [ ]:
import time
import threading
import matplotlib.pyplot as plt
import numpy as np

# k usados no experimento
k_values = [1, 2, 4, 6, 8, 10]

# estrutura para armazenar os tempos de cada experimento
resultados = {
    "A_READ_COMMITTED": [],
    "A_SERIALIZABLE": [],
    "B_READ_COMMITTED": [],
    "B_SERIALIZABLE": []
}

estatisticas_tentativas = []

def executar_experimento(k, versao_func, isolation_level, num_clientes=200):
    """
    Executa um experimento com:
    - k threads (agentes de viagem)
    - num_clientes clientes (default = 200)
    - versao_func: reserva_versaoA ou reserva_versaoB
    - isolation_level: 'READ COMMITTED' ou 'SERIALIZABLE'

    Retorna:
        tempo_total (float),
        tentativas_total (lista de tentativas por cliente),
        estatisticas (dict com Min, Max, Média)
    """

    # Começa sempre com a tabela de 200 assentos livres
    reset_db()

    inicio = time.time()
    tentativas_total = []

    contador = {"cliente": 0}
    lock = threading.Lock()

    def worker():
        while True:
            # pegar "um cliente" para esta thread
            with lock:
                if contador["cliente"] >= num_clientes:
                    return
                contador["cliente"] += 1

            # atende um cliente (uma reserva completa)
            tentativas = versao_func(isolation_level)
            tentativas_total.append(tentativas)

    # cria k threads/agentes
    threads = [threading.Thread(target=worker) for _ in range(k)]

    # inicia e aguarda todas
    for t in threads:
        t.start()
    for t in threads:
        t.join()

    tempo_total = time.time() - inicio

    # ---- estatísticas das tentativas ----
    arr = np.array(tentativas_total)
    estatisticas = {
        "Min": arr.min(),
        "Max": arr.max(),
        "Média": arr.mean()
    }

    return tempo_total, tentativas_total, estatisticas



Executar Experimentos

In [ ]:
for k in k_values:
    # Versão A - Read Committed
    t, tent, stats = executar_experimento(k, reserva_versaoA, "READ COMMITTED")
    resultados["A_READ_COMMITTED"].append(t)

    estatisticas_tentativas.append({
        "k": k,
        "Versão": "A",
        "Isolamento": "READ COMMITTED",
        "Min": stats["Min"],
        "Max": stats["Max"],
        "Média": stats["Média"]
    })


In [ ]:
for k in k_values:
    # Versão A - Serializable
    t, tent, stats = executar_experimento(k, reserva_versaoA, "SERIALIZABLE")
    resultados["A_SERIALIZABLE"].append(t)

    estatisticas_tentativas.append({
        "k": k,
        "Versão": "A",
        "Isolamento": "SERIALIZABLE",
        "Min": stats["Min"],
        "Max": stats["Max"],
        "Média": stats["Média"]
    })


In [ ]:
for k in k_values:
    # Versão B - Read Committed
    t, tent, stats = executar_experimento(k, reserva_versaoB, "READ COMMITTED")
    resultados["B_READ_COMMITTED"].append(t)

    estatisticas_tentativas.append({
        "k": k,
        "Versão": "B",
        "Isolamento": "READ COMMITTED",
        "Min": stats["Min"],
        "Max": stats["Max"],
        "Média": stats["Média"]
    })


In [ ]:
for k in k_values:
    # Versão B - Serializable
    t, tent, stats = executar_experimento(k, reserva_versaoB, "SERIALIZABLE")
    resultados["B_SERIALIZABLE"].append(t)

    estatisticas_tentativas.append({
        "k": k,
        "Versão": "B",
        "Isolamento": "SERIALIZABLE",
        "Min": stats["Min"],
        "Max": stats["Max"],
        "Média": stats["Média"]
    })


Gerar Gráficos

In [ ]:
def gerar_grafico(nome, valores, titulo):
    plt.figure(figsize=(8,5))
    plt.plot(k_values, valores, marker='o')
    plt.xlabel("Número de agentes (k)")
    plt.ylabel("Tempo total (s)")
    plt.title(titulo)
    plt.grid(True)

    # Range fixo para facilitar comparação entre gráficos 
    plt.ylim(0, 300)  

    plt.show()



gerar_grafico(
    "A_READ_COMMITTED",
    resultados["A_READ_COMMITTED"],
    "Versão A – Read Committed"
)

gerar_grafico(
    "A_SERIALIZABLE",
    resultados["A_SERIALIZABLE"],
    "Versão A – Serializable"
)

gerar_grafico(
    "B_READ_COMMITTED",
    resultados["B_READ_COMMITTED"],
    "Versão B – Read Committed"
)

gerar_grafico(
    "B_SERIALIZABLE",
    resultados["B_SERIALIZABLE"],
    "Versão B – Serializable"
)


### Tarefa 19 — Tabela de tentativas

In [ ]:
import pandas as pd

# Criar DataFrame final
df_tentativas = pd.DataFrame(estatisticas_tentativas)

# Ordenar 
df_tentativas = df_tentativas.sort_values(by=["Versão", "Isolamento", "k"])

df_tentativas


### Tarefa 20 - Análise dos Resultados




1. Versão A — Read Committed

Na versão A com Read Committed, todos os clientes concluíram a reserva sempre na primeira tentativa.

(Min = Max = Média = 1)

Isso ocorre porque:

- A leitura, a espera de 1 segundo e a escrita são executadas dentro de uma única transação.

- No nível Read Committed, a leitura não é bloqueada e o SGBD não precisa garantir serialização global.

- Conflitos reais só acontecem na escrita, e a chance de dois agentes reservarem o mesmo assento simultaneamente é baixa.

*Conclusão*: desempenho estável, sem retrabalho e tempos compatíveis com o aumento de k.


2. Versão A — Serializable

Este foi o cenário mais custoso da simulação. O número de tentativas aumentou significativamente conforme k cresceu, chegando a até 8 retries.

Motivos:

- O nível Serializable exige que a transação inteira seja equivalente a uma execução isolada.

- A versão A mantém a transação aberta durante o tempo de espera de 1 segundo, aumentando a janela crítica.

- Vários agentes leem o mesmo estado dos assentos e tentam reservar posições semelhantes.

- O PostgreSQL detecta conflitos de serialização (como phantom reads e write-write conflicts) e aborta transações, obrigando novas tentativas.

*Conclusão:* alta contenção, muitos abortos de transação e pior desempenho entre todas as combinações.

3. Versão B — Read Committed

A versão B apresentou apenas uma tentativa por cliente em todos os valores de k.

Isso acontece porque:

- A leitura e a escrita acontecem em transações separadas, ambas muito curtas.

- O tempo de espera de 1 segundo ocorre fora de qualquer transação, reduzindo drasticamente a chance de conflito.

- A escrita é rápida o suficiente para não gerar bloqueios significativos.

*Conclusão:* excelente desempenho e nenhuma tentativa extra, mesmo com alta concorrência.

4. Versão B — Serializable

Mesmo sob Serializable, a versão B manteve Min = Max = Média = 1.
Ou seja, a serialização não gerou overhead significativo.

Razões:

- Como a transação de escrita é extremamente curta, as chances de conflito são mínimas.

- A separação entre leitura e escrita impede phantoms e reduz o risco de abortos.

*Conclusão:* comportamento extremamente eficiente, mostrando que a versão B é robusta mesmo no nível de isolamento mais forte.